In [14]:
from bs4 import BeautifulSoup
from collections import Counter, defaultdict
import json
import seaborn as sns
from tqdm import tqdm

In [2]:
with open("data/href2dramapage.json", encoding="utf-8") as f:
    href2dramapage = json.load(f)

In [27]:
# href2details = defaultdict(dict)

for href, page in tqdm(href2dramapage.items()):
    if href not in href2details:
        
        soup = BeautifulSoup(page)
        details = soup.find_all("ul", {"class": "list m-b-0"})
        if details:
            
            for details_block in details:
                separate_details = details_block.find_all("li", {"class": "list-item p-a-0"})
                
                for separate_detail in separate_details:
                    
                    detail_prefix = separate_detail.find_all("b", {"class": "inline"})
                    assert len(detail_prefix) == 1
                    
                    detail_key = detail_prefix[0].get_text()
                    detail_value = separate_detail.get_text()
                    assert detail_value.startswith(detail_key)
                    
                    assert detail_key not in href2details[href]
                    href2details[href][detail_key] = detail_value

100%|████████████████████████████████████████████████████████████████████████████| 19462/19462 [30:23<00:00, 10.67it/s]


In [28]:
# %%time
# with open("data/href2details.json", "w", encoding="utf-8") as fw:
#     json.dump(href2details, fw, ensure_ascii=False)

CPU times: total: 922 ms
Wall time: 956 ms


In [30]:
len(href2dramapage)

19462

In [34]:
[href for href in href2dramapage if href not in href2details]

['/697659-big-mouth',
 '/697661-big-mouth',
 '/28640-see-you-again',
 '/744221-pretty-guardian-of-the-city',
 '/22041-magic-journey-to-africa',
 '/54603-run-bts-season-4',
 '/730725-reverse',
 '/716857-floor',
 '/32134-sanctuary',
 '/712575-shimobee',
 '/26061-faded',
 '/706905-the-remarried-empress',
 '/68071-beats-per-mizu',
 '/714027-the-hidden-fox',
 '/77971-off-gun-fun-night-season-2-special',
 '/23766-four-men',
 '/27100-dong-witch-and-me',
 '/685235-mentalist',
 '/20150-soulmates-the-series',
 '/22147-housewife-detective',
 '/17067-blue-in-my-heart',
 '/32734-hot-place',
 '/53895-wet-season',
 '/32769-younger',
 '/683333-triple-do-you-want',
 '/701285-last-twilight-in-phuket',
 '/682103-kirei-kamisama-to-machiawaseshita-onna',
 '/732917-cang-lan-jue',
 '/715703-cherry-maho-the-movie',
 '/15959-battle-of-nujiang',
 '/18836-angels-revenge',
 '/732821-new-untitled-sbs-reality-program',
 '/730863-we-are-the-champions',
 '/28343-saja',
 '/26952-2015-dream-concert',
 '/21912-the-happi

In [32]:
Counter([k for d in href2details.values() for k in d.keys()]).most_common()

[('Country:', 19393),
 ('Score:', 19393),
 ('Ranked:', 19393),
 ('Popularity:', 19393),
 ('Watchers:', 19393),
 ('Favorites:', 19393),
 ('Duration:', 18055),
 ('Episodes:', 13028),
 ('Aired:', 12258),
 ('Original Network:', 10738),
 ('Aired On:', 10189),
 ('Drama:', 9735),
 ('Movie:', 6365),
 ('Release Date:', 6365),
 ('TV Show:', 2101),
 ('Special:', 1192),
 ('Airs:', 770),
 ('Airs On:', 47)]

In [16]:
separate_details[0]

<li class="list-item p-a-0"><b class="inline">Score:</b> 8.2 <span class="hft">(scored by 12 users)</span></li>

In [17]:
details[1]

<ul class="list m-b-0"> <li class="list-item p-a-0"><b class="inline">Score:</b> 8.2 <span class="hft">(scored by 12 users)</span></li> <li class="list-item p-a-0"><b class="inline">Ranked:</b> #37553</li> <li class="list-item p-a-0"><b class="inline">Popularity:</b> #12666</li> <li class="list-item p-a-0"><b class="inline">Watchers:</b> 302</li> <li class="list-item p-a-0" style="display:none;"><b class="inline">Favorites:</b> 0</li> </ul>

In [3]:
with open("data/href2details.json", encoding="utf-8") as f:
    href2details = json.load(f)

In [4]:
import pandas as pd

df = pd.DataFrame({"details": href2details.values()})
df = pd.json_normalize(df["details"])
df["href"] = href2details.keys()

In [5]:
for col in df.columns:
    if col.endswith(":"):
        df[col] = df[col].apply(
            lambda x: (x[len(col):].strip() if (type(x) == str) else x)
        )

In [6]:
df["Watchers:"] = df["Watchers:"].apply(
    lambda x: int(x.replace(",", ""))
)

In [7]:
df["scorers"] = df["Score:"].apply(
    lambda x: int(x.split()[-2].replace(",", ""))
)
df["Score"] = df["Score:"].apply(
    lambda x: x.split()[0]
)

df["Score"] = df["Score"].apply(
    lambda x: None if (x == 'N/A') else float(x)
)

In [9]:
cols = ["Country:"]
df.groupby(cols).agg({
    "Watchers:": ["median", "mean"]
    , "Original Network:": ["nunique"]
    , "Score": ["mean", "median"]
}).sort_values(("Watchers:", "median"))

Watchers:              Original Network:     Score       
               median         mean           nunique      mean median
Country:                                                             
Philippines     114.5   556.150000                 9  7.444240    7.5
Hong Kong       202.0   390.447059                16  7.365303    7.5
Japan           350.0  1232.555932                80  7.424339    7.5
China           464.0  1940.104172               171  7.610948    7.6
Thailand        611.0  2523.874016                40  7.482297    7.5
Taiwan          621.0  2339.784257                59  7.394562    7.4
South Korea     771.0  5113.669519               137  7.731071    7.7

In [8]:
def duration_to_minutes(s: str) -> int:
    if type(s) != str:
        return s
    lst = s.split()
    if len(lst) == 2:
        return int(lst[0])
    if len(lst) == 4:
        return (int(lst[0]) * 60) + int(lst[2])
    return s

df["Duration"] = df["Duration:"].apply(duration_to_minutes)

In [12]:
df[
    df["Drama:"].notna()
].groupby("Country:").agg({"Duration": ["mean", "median"]}).sort_values(("Duration", "mean"))

Duration       
                  mean median
Country:                     
Philippines  28.649533   30.0
China        37.559842   45.0
Japan        39.161027   45.0
Hong Kong    44.080508   45.0
South Korea  47.269392   60.0
Taiwan       59.995327   60.0
Thailand     67.752657   60.0

In [9]:
df["Episodes"] = df["Episodes:"].apply(
    lambda x: int(x) if (type(x) == str) else x
)

In [10]:
df["total_duration"] = [
    t.Duration * t.Episodes
    if (t.Duration and t.Episodes)
    else None
    for t in df.itertuples()
]

In [11]:
df["Popularity"] = df["Popularity:"].apply(
    lambda x: int(x[1:]) if (x != "#99999") else None
)

In [12]:
df["Ranked"] = df["Ranked:"].apply(
    lambda x: int(x[1:]) if (x != "#99999") else None
)

In [13]:
df[["Score", "Duration", "Episodes", "total_duration", "scorers", "Popularity", "Ranked"]].corr()

,Score,Duration,Episodes,total_duration,scorers,Popularity,Ranked
Score,1.000000,-0.078422,0.038440,0.025866,0.173525,-0.231818,-0.352680
Duration,-0.078422,1.000000,0.023807,0.070518,-0.006726,0.020296,-0.017381
Episodes,0.038440,0.023807,1.000000,0.886674,-0.014250,0.027139,0.024601
total_duration,0.025866,0.070518,0.886674,1.000000,-0.005888,0.005096,0.010465
scorers,0.173525,-0.006726,-0.014250,-0.005888,1.000000,-0.422350,-0.296139
Popularity,-0.231818,0.020296,0.027139,0.005096,-0.422350,1.000000,0.749527
Ranked,-0.352680,-0.017381,0.024601,0.010465,-0.296139,0.749527,1.000000


In [16]:
tmp = df.groupby("Country:").agg({
    col: "median" for col 
    in ["Score", "Duration", "Episodes", "total_duration", "scorers", "Popularity", "Ranked"]
})

tmp

,Score,Duration,Episodes,total_duration,scorers,Popularity,Ranked
Country:,,,,,,,
China,7.6,45.0,26.0,1200.0,70.0,6606.5,34464.0
Hong Kong,7.5,90.0,25.0,1125.0,85.0,10568.5,44142.5
Japan,7.5,55.0,10.0,360.0,110.0,7885.0,40286.0
Philippines,7.5,45.0,10.0,280.0,32.5,8984.5,40810.5
South Korea,7.7,65.0,12.0,560.0,227.0,5273.0,7526.0
Taiwan,7.4,70.0,15.0,1163.0,168.0,6711.0,8522.0
Thailand,7.5,60.0,12.0,782.0,131.0,6321.0,34008.0


In [19]:
# sns.heatmap((tmp-tmp.mean())/tmp.std(), annot=True);

In [27]:
df["Aired On:"].value_counts().head(11)

Friday                                                            1283
Saturday                                                           999
Sunday                                                             984
Thursday                                                           761
Monday, Tuesday                                                    660
Wednesday, Thursday                                                624
Monday, Tuesday, Wednesday, Thursday, Friday                       615
Wednesday                                                          597
Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday     595
Tuesday                                                            539
Monday                                                             522
Name: Aired On:, dtype: int64